# Base62 vs UUID ids

For more background on this topic and the compute, see [lamin.ai/notes/2022/ids](https://lamin.ai/notes/2022/ids).

In [ ]:
from uuid import UUID
import base64
from lnschema_core import id

Here are a few UUIDs.

In [ ]:
uuids = [
    "a59c5072d1c1474baa34870b46a24c77",
    "7ff09b2c075e45bba68b227929f9f7e3",
    "b1c40fc23bec419e9eb3497e2a50beed",
    "fec1cfdefc5d4385bf078e946c802a88",
]

For user IDs, we choose 8 base62 characters.

This is enough to store 2e+14 IDs and has these collision probabilities (birthday problem) for 10k, 100k, and 1M users respectively:
```
n = 1e+04: p = 2e-07
n = 1e+05: p = 2e-05
n = 1e+06: p = 2e-03
n = 1e+07: p = 2e-01
n = 1e+08: p = 1e+00
```

Hence, even at 999 999 users, the chance that user #1M registers a colliding ID is only 2e-03.

We're checking for uniqueness of these IDs upon user registration, and we'll likely not need to catch an error until we're beyond 1M users.

At first, we thought that a deterministic compute might as below might be cool as it allows to bypass a query if one has the UUID. 

In [ ]:
def base62_from_uuid(uuid: UUID, n_char: int):
    """Truncated and base62-mapped UUID.

    Straight-forward convert from UUID to base62.
    The reverse is not true as information is not conserved!
    """
    uuid_b64 = base64.urlsafe_b64encode(uuid.bytes).decode("ascii")
    base64_truncate = uuid_b64[:n_char]
    # the following will slightly favor 0 and 1 and therefore entropy
    # isn't maximal in base62, but that's OK for us
    base62_truncate = base64_truncate.replace("_", "0").replace("-", "1")
    return base62_truncate

In [ ]:
lnids = [base62_from_uuid(UUID(uuid), n_char=8) for uuid in uuids]

In [ ]:
lnids

In [ ]:
assert lnids == ["pZxQctHB", "f0CbLAde", "scQPwjvs", "0sHP3vxd"]

However, the query for the UUID in user table is always going to be fast, hence, we can also just go with a randomly generated 8 character base62 ID, as for all other entities.

The random ID has the advantage that we can easily generated a few in case the uniqueness constraint fails every now and then at 1M users. And at 100M users, we can either use a smart suggestion for available IDs or expand to 9 characters. 😅

Hence, we'll just generate a random 8-char base62 along with the UUID for each new user.

In [ ]:
random_lnids = [id.id_user() for i in range(4)]

In [ ]:
random_lnids